In [6]:
library(repr)
library(tidyverse)
library(tidymodels)
library(readxl)
library(janitor)

In [14]:
tennis_data <- read_csv("Data/player_stats.csv", skip = 1)

tennis_df <- tennis_data |>
    clean_names() |>
    select(age, current_rank, prize_money, height, seasons) |>
    separate(col = height, 
            into = c("height_cm", "discard"), 
           sep = " ") |>
    separate(col = current_rank, 
            into = c("rank", "discard_1"), 
           sep = " ") |>
    separate(col = age, 
            into = c("age", "discard_2"), 
           sep = " ") |>
    select(-discard_2, -discard_1, -discard) |>
    mutate(age = as.numeric(age),
           rank = as.numeric(rank),
           height_cm = as.numeric(height_cm))

means <- tennis_df |>
            select(age:seasons) |>
            map_dfr(mean, na.rm = TRUE)

tennis_df <- tennis_df |>
                replace_na(list(age = 25.96794, 
                                prize_money = 3416440, 
                                height_cm = 185.7913, 
                                seasons = 6.494652)) |>
    arrange(rank)
    
tennis_df_rank <- tennis_df |>
    select(rank)

rank_quantile <- quantile(tennis_df_rank, probs = c(0.25, 0.5, 0.75), na.rm = TRUE)


rank_quantile
means
tennis_df

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (24): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (14): ...1, Prize Money, Turned Pro, Seasons, Titles, Best Season, Retir...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


25%   50%   75% 
125.5 249.0 372.5

age,rank,prize_money,height_cm,seasons
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
25.96794,249.0283,3416440,185.7913,6.494652


age,rank,prize_money,height_cm,seasons
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
33,1,119601561,185.0000,19
32,2,139144944,188.0000,17
38,3,129231891,185.0000,22
23,4,10507693,185.7913,5
26,5,22132368,185.0000,10
21,6,10425605,185.7913,4
22,7,20028563,198.0000,8
23,8,3580862,185.7913,3
31,9,12067808,183.0000,11
